In [2]:
# Libraries
from scipy.stats import qmc
import numpy as np
import copy
import csv
import pandas as pd
import os
import netCDF4 as nc4
import sys
import shutil
from tempfile import TemporaryFile                                                                                                                                 
import argparse                                                                                                                                                                                                                                                                                                       
import tempfile 
import random
import re
import time as tm

import modp as mp

### Constants and options

In [1]:
# Number of members for revising existing parameter files
nens = 638

# Number of LHC sampling for producing parameters
n_inst = 1600

# Random seed number
seedno = 32

### Generate PPE parameters

In [ ]:
# Read in min and max values for each parameter and pft 
param_ranges_full = pd.read_csv('/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/sr_ensemble_params.csv')
param_ranges = param_ranges_full[['param', 'value_mean', 'value_min', 'value_max', 'pft', 'organ']]

# number of parameters
n_params = len(param_ranges)

# number of PFTs - some are global so subtract one
n_pfts = len(pd.unique(param_ranges['pft'])) - 1

param_names = list(param_ranges.param)
pfts = list(param_ranges.pft)
organs = list(param_ranges.organ)

print(param_ranges)

In [ ]:
sampler = qmc.LatinHypercube(d=n_params)
sample = sampler.random(n=n_inst)

# scale to parameter ranges
l_bounds = param_ranges['value_min']
u_bounds = param_ranges['value_max']

scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

print(scaled_sample[0,:])

In [5]:
# Read in defaut FATES parameter file and produce new files with perturbed parameters
# settle random seed for reproducibility
random.seed(seedno)
input_fname = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/fates_params_sr_base.nc'

# for each sample 
for i in range(0,n_inst) :
    
    ens_num = str(i+1).zfill(4)
    # final parameter file name
    fout = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/fates_params_sr_ens_'+ens_num+'_caldirlog.nc'
    
    shutil.copyfile(input_fname, fout)                                                                                                                             
   
    # loop through each parameter and apply either to the correct pft or globally
    for j in range(0, n_params) : 
        
        var = param_names[j]
        pft = pfts[j]
        organ = organs[j]
        
        val = scaled_sample[i, j]
        
        mp.main(var = var, pft = pft, fin = fout, val = val, 
                    fout = fout, O = 1, organ = organ)
        
#         if var == 'fates_mort_bmort' and pft == 1 : 
#             pft = pft + 1
#             val = val * 0.75
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)
            
#         if var == 'fates_wood_density' and pft == 1 : 
#             pft = pft + 1
#             val = val * 1.5
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)
            
#         if var == 'fates_leaf_vcmax25top' and pft == 1 : 
#             pft = pft + 1
#             val = val * 0.9
#             mp.main(var = var, pft = pft, fin = fout, val = val, 
#                     fout = fout, O = 1, organ = organ)

    # - certain parameters have constraints so we apply them outside the parameter loop
    # parameters constrained by the sum equals to one
    idx_frflig = [k for k, x in enumerate(param_names) if x == 'fates_frag_fnrt_flig']
    idx_frfcel = [k for k, x in enumerate(param_names) if x == 'fates_frag_fnrt_fcel']
    idx_lfflig = [k for k, x in enumerate(param_names) if x == 'fates_frag_leaf_flig']
    idx_lffcel = [k for k, x in enumerate(param_names) if x == 'fates_frag_leaf_fcel']
    idx_cwfcel = param_names.index('fates_frag_cwd_fcel')
    
    val_frflab = 1.0 - scaled_sample[i, idx_frflig[0]] - scaled_sample[i, idx_frfcel[0]]
    mp.main(var = 'fates_frag_fnrt_flab', pft = pfts[idx_frflig[0]], fin = fout, val = val_frflab, 
                    fout = fout, O = 1, organ = organs[idx_frflig[0]])

    val_frflab = 1.0 - scaled_sample[i, idx_frflig[1]] - scaled_sample[i, idx_frfcel[1]]
    mp.main(var = 'fates_frag_fnrt_flab', pft = pfts[idx_frflig[1]], fin = fout, val = val_frflab, 
                    fout = fout, O = 1, organ = organs[idx_frflig[1]])
    
    val_lfflab = 1.0 - scaled_sample[i, idx_lfflig[0]] - scaled_sample[i, idx_lffcel[0]]
    mp.main(var = 'fates_frag_leaf_flab', pft = pfts[idx_lfflig[0]], fin = fout, val = val_lfflab, 
                    fout = fout, O = 1, organ = organs[idx_lfflig[0]])

    val_lfflab = 1.0 - scaled_sample[i, idx_lfflig[1]] - scaled_sample[i, idx_lffcel[1]]
    mp.main(var = 'fates_frag_leaf_flab', pft = pfts[idx_lfflig[1]], fin = fout, val = val_lfflab, 
                    fout = fout, O = 1, organ = organs[idx_lfflig[1]])
    
    val_cwflig = 1.0 - scaled_sample[i, idx_cwfcel]
    mp.main(var = 'fates_frag_cwd_flig', pft = 0, fin = fout, val = val_cwflig, 
                    fout = fout, O = 1, organ = 0)
    

### Revise the logging related parameter files

In [3]:
## in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

# list_can = np.array([  18,  135,  267,  291,  331,  352,  386,  420,  463,  471,  519,
#         541,  567,  834,  883,  916,  938,  994, 1006, 1068, 1241, 1246,
#        1374, 1431, 1500, 1530, 1580])-1
#list_can = np.array([386])-1


for i in np.arange(0,nens):
    
    ens_num = str(i+1).zfill(4)
    # final parameter file name
    fout = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/uav_mx_breeding/fates_params_sr_ens_'+ens_num+'.nc'

    mp.main(var = 'fates_landuse_logging_coll_under_frac', fin = fout, val = 0.55983, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    mp.main(var = 'fates_landuse_logging_collateral_frac', fin = fout, val = 0.0, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    mp.main(var = 'fates_landuse_logging_dbhmin', fin = fout, val = 10.0, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    mp.main(var = 'fates_landuse_logging_dbhmax', fin = fout, val = 50.0, 
                    fout = fout, O = 1, pft=0, organ=0)    
    
    mp.main(var = 'fates_landuse_logging_direct_frac', fin = fout, val = 0.3, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    mp.main(var = 'fates_landuse_logging_export_frac', fin = fout, val = 1.0, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    mp.main(var = 'fates_landuse_logging_mechanical_frac', fin = fout, val = 0.0, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    # # loop through each parameter and apply either to the correct pft or globally
    # mp.main(var = 'fates_landuse_logging_collateral_frac', fin = fout, val = 0.02, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_direct_frac', fin = fout, val = 0.95, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_mechanical_frac', fin = fout, val = 0.03, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_dbhmin', fin = fout, val = 30, 
    #                 fout = fout, O = 1, pft=0, organ=0)
 

### Revise allometric parameters

In [4]:
## in defaut FATES file - note that this is the default for FATES but with:
# - updated allometries for tropical PFTs
# - size bins that are consistent with the DBEN protocol. 
# - supplemental seed rain
# - updated vai bins
# - atkin respiration 
# - size dependent mortality 

# list_can = np.array([  18,  135,  267,  291,  331,  352,  386,  420,  463,  471,  519,
#         541,  567,  834,  883,  916,  938,  994, 1006, 1068, 1241, 1246,
#        1374, 1431, 1500, 1530, 1580])-1
#list_can = np.array([386])-1

for i in np.arange(0,nens):
    
    ens_num = str(i+1).zfill(4)
    # final parameter file name
    fout = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/uav_mx_breeding/fates_params_sr_ens_'+ens_num+'.nc'

    mp.main(var = 'fates_allom_d2h1', fin = fout, val = 1.50E+07, 
                    fout = fout, O = 1, pft=1, organ=0)
    
    mp.main(var = 'fates_allom_d2h2', fin = fout, val = 4.14E-01, 
                    fout = fout, O = 1, pft=1, organ=0)
    
    mp.main(var = 'fates_allom_d2h3', fin = fout, val = 4.77E+06, 
                    fout = fout, O = 1, pft=1, organ=0)

#     mp.main(var = 'fates_allom_blca_expnt_diff', fin = fout, val = -0.94, 
#                     fout = fout, O = 1, pft=1, organ=0)
        
#     mp.main(var = 'fates_allom_d2ca_coefficient_min', fin = fout, val = 3.03, 
#                     fout = fout, O = 1, pft=1, organ=0)

#     mp.main(var = 'fates_allom_d2ca_coefficient_max', fin = fout, val = 3.03, 
#                     fout = fout, O = 1, pft=1, organ=0)
    
    mp.main(var = 'fates_allom_blca_expnt_diff', fin = fout, val = -0.2307200846, 
                    fout = fout, O = 1, pft=1, organ=0)
        
    mp.main(var = 'fates_allom_d2ca_coefficient_min', fin = fout, val = 0.59158659, 
                    fout = fout, O = 1, pft=1, organ=0)

    mp.main(var = 'fates_allom_d2ca_coefficient_max', fin = fout, val = 0.59158659, 
                    fout = fout, O = 1, pft=1, organ=0)
       
    mp.main(var = 'fates_allom_d2h1', fin = fout, val = 5.59E+07, 
                    fout = fout, O = 1, pft=2, organ=0)    
    
    mp.main(var = 'fates_allom_d2h2', fin = fout, val = 6.96E-01, 
                    fout = fout, O = 1, pft=2, organ=0)
    
    mp.main(var = 'fates_allom_d2h3', fin = fout, val = 3.23E+07, 
                    fout = fout, O = 1, pft=2, organ=0)

#     mp.main(var = 'fates_allom_blca_expnt_diff', fin = fout, val = -0.53, 
#                     fout = fout, O = 1, pft=2, organ=0)
        
#     mp.main(var = 'fates_allom_d2ca_coefficient_min', fin = fout, val = 1.09, 
#                     fout = fout, O = 1, pft=2, organ=0)

#     mp.main(var = 'fates_allom_d2ca_coefficient_max', fin = fout, val = 1.09, 
#                     fout = fout, O = 1, pft=2, organ=0)

    mp.main(var = 'fates_allom_blca_expnt_diff', fin = fout, val = -0.248765948, 
                    fout = fout, O = 1, pft=2, organ=0)
        
    mp.main(var = 'fates_allom_d2ca_coefficient_min', fin = fout, val = 0.41284662, 
                    fout = fout, O = 1, pft=2, organ=0)

    mp.main(var = 'fates_allom_d2ca_coefficient_max', fin = fout, val = 0.41284662, 
                    fout = fout, O = 1, pft=2, organ=0)

    mp.main(var = 'fates_vai_width_increase_factor', fin = fout, val = 1.1, 
                    fout = fout, O = 1, pft=0, organ=0)
    
    # # loop through each parameter and apply either to the correct pft or globally
    # mp.main(var = 'fates_landuse_logging_collateral_frac', fin = fout, val = 0.02, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_direct_frac', fin = fout, val = 0.95, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_mechanical_frac', fin = fout, val = 0.03, 
    #                 fout = fout, O = 1, pft=0, organ=0)
    # mp.main(var = 'fates_landuse_logging_dbhmin', fin = fout, val = 30, 
    #                 fout = fout, O = 1, pft=0, organ=0)
 

### Revise landuse time series file for C-based harvest, post-1980

In [9]:
# Two options of harvest intensity, 0.3 maybe closer to the reality
dir_frac = 0.3
# dir_frac = 1.0
use_hrv_dbh = True
hrv_dbh_min = 10
hrv_dbh_max = 50

# SR site data
tot_forest_area = 1920.0 # ha
tot_land_area = 3200.0 # ha

# The year in land use time series is one year after the model simulation year.
lu_year = np.arange(1981, 2029)

# Some harvest rate data from reports
# Calculated by adding harvested volumes of all 4 species 
hrvv_obs = np.array([8726.77, 8250.93, 10010.56, 8308.78, 8751.57, 6601.09, 9822.54, 6610.51, 4716.57, 421.84, 4971.61, 5274.91, 5785.58, 5124.30, 
                   6866.16, 8509.43, 9922.54, 9749.23, 9593.83, 8741.28])
hrvv_pre2008 = 7978.0 * np.ones((2008-1980))
hrvv_baseline = 7978.0 * np.ones((2028-1980))
# hrvv_o = np.concatenate((hrvv_pre2008, hrvv_obs))
hrvv_o = hrvv_baseline

hrvv = hrvv_o / tot_forest_area

# Open and collect wood density information from candidates within the ensemble
# Path to all members
wd = []
param_prefix = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/uav_mx_breeding/fates_params_sr_ens_'
for i in np.arange(0,nens):
    ens_num = str(i+1).zfill(4)
    fpath = param_prefix+ens_num+'.nc'
    dtc = nc4.Dataset(fpath)
    tmp = dtc['fates_wood_density'][:]
    wd.append(tmp[0])
    dtc.close()

# from g cm-3 to kg m-3
arr_wd = 1000.0 * np.array(wd)

del tmp

# Calculate the harvested C amount
hrvc = []
for i in np.arange(0,nens):
    arr_hrvc = hrvv * arr_wd[i] * 0.5
    hrvc.append(arr_hrvc)



In [10]:
# Obtain total biomass from restart file to calculate the rotation length
# Open the file and obtain the last value
# This step will take much longer than expected.
start = tm.time()

get_mean = False
# Path to all ensemble members
# block_path = '/pscratch/sd/s/sshu3/FATES_MRV/fates_mxbreeding254_spinup_2024-07-26_e9515ed7a8_664db78d/run/'
# block_prefix = 'fates_mxbreeding254_spinup_2024-07-26_e9515ed7a8_664db78d.elm_'
# block_path = '/pscratch/sd/s/sshu3/FATES_MRV/fates_mxbreeding254_spinup_2024-08-29_e9515ed7a8_d8f0eb12/run/'
# block_prefix = 'fates_mxbreeding254_spinup_2024-08-29_e9515ed7a8_d8f0eb12.elm_'
# block_path = '/pscratch/sd/s/sshu3/FATES_MRV/fates_mxbreeding254_spinup_2024-05-27_e9515ed7a8_664db78d/run/'
# block_prefix = 'fates_mxbreeding254_spinup_2024-05-27_e9515ed7a8_664db78d.elm_'
# fyear = [161, 321, 481]

# New candidates based on UAV LiDAR
block_n = 3
block_ens = [210, 210, 218]

# Path to all ensemble members
block_path = ['/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_1_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/', \
              '/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_2_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/', \
              '/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_3_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/']
block_prefix = ['fates_uav_mxbreeding638_1_spinup_2024-10-26_e9515ed7a8_d8f0eb12.elm_',\
               'fates_uav_mxbreeding638_2_spinup_2024-10-26_e9515ed7a8_d8f0eb12.elm_', \
               'fates_uav_mxbreeding638_3_spinup_2024-10-26_e9515ed7a8_d8f0eb12.elm_']
fyear = [1970] 

# Constants
nyr = len(fyear)
nens = 638
abg_frac = 0.6

# Initialize as blank lists
cnplant_collect = []
cheight_collect = []
clai_collect = []
ccarea_collect = []
cleafc_collect = []
cfnrtc_collect = []
csapwc_collect = []
cstorc_collect = []
creproc_collect = []
cstruc_collect = []
cdbh_collect = []
cpft_collect = []

# Loop through members and collect the total available carbon for harvest
# since cohort number is quite different from each time snapshot, we shall use list
# but to my surprise, all files seem to have a fixed number of cohorts = 6400
# Think this as the maximum number and unused cohorts may exit (though it seems not to be the case) in each file.
for k in np.arange(0,block_n):
    for i in np.arange(0,block_ens[k]):
        for j in np.arange(0, len(fyear)):
            ens_num = str(i+1).zfill(4)
            fpath = block_path[k]+block_prefix[k]+ens_num+'.r.'+str(fyear[j]).zfill(4)+'-01-01-00000.nc'
            dtc = nc4.Dataset(fpath)
            dnc_per_patch = dtc['fates_CohortsPerPatch'][:]
            dnplant = dtc['fates_nplant'][:]
            dheight = dtc['fates_height'][:]
            dlai = dtc['fates_cohort_treelai'][:]
            dcarea = dtc['fates_cohort_area'][:]
            dleafc = dtc['leaf_c_val_001'][:]
            dfnrtc = dtc['fnrt_c_val_001'][:]
            dsapwc = dtc['sapw_c_val_001'][:]
            dstorc = dtc['store_c_val_001'][:]
            dreproc = dtc['repro_c_val_001'][:]
            dstruc = dtc['struct_c_val_001'][:]
            ddbh = dtc['fates_dbh'][:]
            dpft = dtc['fates_pft'][:]
            dtc.close()

            ncohorts = len(dnc_per_patch)

            # Concatenate into same time series
            if(j == 0):
                dnplant_comb = [copy.deepcopy(dnplant)]
                dheight_comb = [copy.deepcopy(dheight)]
                dlai_comb = [copy.deepcopy(dlai)]
                dcarea_comb = [copy.deepcopy(dcarea)]
                dleafc_comb = [copy.deepcopy(dleafc)]
                dfnrtc_comb = [copy.deepcopy(dfnrtc)]
                dsapwc_comb = [copy.deepcopy(dsapwc)]
                dstorc_comb = [copy.deepcopy(dstorc)]
                dreproc_comb = [copy.deepcopy(dreproc)]
                dstruc_comb = [copy.deepcopy(dstruc)]
                ddbh_comb = [copy.deepcopy(ddbh)]
                dpft_comb = [copy.deepcopy(dpft)]
            else:
                dnplant_comb.append(copy.deepcopy(dnplant)[:])
                dheight_comb.append(copy.deepcopy(dheight)[:])
                dlai_comb.append(copy.deepcopy(dlai)[:])
                dcarea_comb.append(copy.deepcopy(dcarea)[:])
                dleafc_comb.append(copy.deepcopy(dleafc)[:])
                dfnrtc_comb.append(copy.deepcopy(dfnrtc)[:])
                dsapwc_comb.append(copy.deepcopy(dsapwc)[:])
                dstorc_comb.append(copy.deepcopy(dstorc)[:])
                dreproc_comb.append(copy.deepcopy(dreproc)[:])
                dstruc_comb.append(copy.deepcopy(dstruc)[:])
                ddbh_comb.append(copy.deepcopy(ddbh)[:])
                dpft_comb.append(copy.deepcopy(dpft)[:])

        # Obtain the time series
        cnplant_collect.append(dnplant_comb)
        cheight_collect.append(dheight_comb)
        clai_collect.append(dlai_comb)
        ccarea_collect.append(dcarea_comb)
        cleafc_collect.append(dleafc_comb)
        cfnrtc_collect.append(dfnrtc_comb)
        csapwc_collect.append(dsapwc_comb)
        cstorc_collect.append(dstorc_comb)
        creproc_collect.append(dreproc_comb)
        cstruc_collect.append(dstruc_comb)
        cdbh_collect.append(ddbh_comb)
        cpft_collect.append(dpft_comb)

# Transfer all lists to array 
arr_cnplant = np.array(cnplant_collect)
arr_cheight = np.array(cheight_collect)
arr_clai = np.array(clai_collect)
arr_ccarea = np.array(ccarea_collect)
arr_cleafc = np.array(cleafc_collect)
arr_cfnrtc = np.array(cfnrtc_collect)
arr_csapwc = np.array(csapwc_collect)
arr_cstorc = np.array(cstorc_collect)
arr_creproc = np.array(creproc_collect)
arr_cstruc = np.array(cstruc_collect)
arr_cdbh = np.array(cdbh_collect)
arr_cpft = np.array(cpft_collect)

arr_wood_c = abg_frac * (arr_csapwc + arr_cstorc  + arr_cstruc) * arr_cnplant
# arr_abg_c = (arr_cleafc + abg_frac * (arr_csapwc + arr_cstorc  + arr_cstruc)) * arr_cnplant
if(use_hrv_dbh):
    arr_csapwc[arr_cdbh<hrv_dbh_min] = 0.0
    arr_csapwc[arr_cdbh>hrv_dbh_max] = 0.0
    arr_cstruc[arr_cdbh<hrv_dbh_min] = 0.0
    arr_cstruc[arr_cdbh>hrv_dbh_max] = 0.0    
    arr_hrv_c = np.nansum(0.67 * dir_frac * abg_frac * (arr_csapwc + arr_cstruc) * arr_cnplant,  axis = 2)
else:
    arr_hrv_c = np.nansum(0.67 * dir_frac * abg_frac * (arr_csapwc + arr_cstruc) * arr_cnplant,  axis = 2)

end = tm.time()
print('Time elapsed: ', end - start)


Time elapsed:  23.328842401504517


In [11]:
# Estimate rotation length for each member.
rot_len = np.ceil(arr_hrv_c[:,0]/np.array(hrvc)[:,0])

# Determine the start of harvest year from the estimated rotation length
vh1_ts = np.zeros((nens, 2029-1850))
sh1_ts = np.zeros((nens, 2029-1850))

for i in np.arange(0,nens):
    # Pre-2008 harvest rate
    if(rot_len[i] >= 28):
        vh1_ts[i, (2009-1850)-int(rot_len[i]):(2009-1850)] = hrvc[i][0]
    else:
        vh1_ts[i, (1981-1850):(1981-1850)+int(rot_len[i])] = hrvc[i][0]
        sh1_ts[i, (1981-1850)+int(rot_len[i]):(2009-1850)] = hrvc[i][0]
    # Post-2008 harvest rate, for harvest rate there's a 1 year lag between data and FATES model
    sh1_ts[i, (2009-1850):(2029-1850)] = hrvc[i][(2009-1981):(2029-1981)]

In [20]:
# Create land use dataset for each candidate (C-based)
fpath = ['/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_1_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/', \
         '/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_2_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/', \
         '/pscratch/sd/s/sshu3/FATES_MRV/fates_uav_mxbreeding638_3_spinup_2024-10-26_e9515ed7a8_d8f0eb12/run/']
param_num = 0
for k in np.arange(0, block_n):
    for i in np.arange(0, block_ens[k]):
        ens_num = str(i+1).zfill(4)
        # Note: Time dimension in base land use timeseries need to be the recording dimension
        tfile = 'landuse.timeseries_0.125x0.125_hist_simyr1850-2028.SanRafael.c231020_'+ens_num+'_baseline.nc'
        cmd_line = 'cp '+fpath[k]+'landuse.timeseries_0.125x0.125_hist_simyr1850-2028.SanRafael.c231020.nc '+fpath[k]+tfile
        os.system(cmd_line)
        dtc = nc4.Dataset(fpath[k]+tfile, 'r+', clobber=True)
        dtc['HARVEST_SH1'][:,0,0] = sh1_ts[param_num,:]
        dtc['HARVEST_SH2'][:,0,0] = 0.0
        dtc['HARVEST_SH3'][:,0,0] = 0.0
        dtc['HARVEST_VH1'][:,0,0] = vh1_ts[param_num,:]
        dtc['HARVEST_VH2'][:,0,0] = 0.0
        dtc['GRAZING'][:,0,0] = 0.0
        dtc.close()
        # Next parameter file
        param_num += 1


### Revise harvest rate time series file from 2008 to apply actual harvest rate

In [14]:
# number of members
nens = 254

# SR site data
tot_forest_area = 1920.0 # ha
tot_land_area = 3200.0 # ha

# The year in land use time series is one year after the model simulation year.
lu_year = np.array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 
                 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028])
# Calculated from surface (i.e., total area harvested ) / total area
hrva_o = np.array([0.047, 0.038, 0.054, 0.04, 0.058, 0.061, 0.12, 0.055, 0.033, 0.039, 
                   0.043, 0.042, 0.036, 0.035, 0.037, 0.041, 0.042, 0.042, 0.044, 0.044])
# Calculated by adding harvested volumes of all 4 specis 
hrvv_o = np.array([10228.48, 8833.41, 11552.41, 9258.62, 10311.47, 12051.44, 10814.60, 
                   8400.84, 8349.92, 8910.00, 7314.93, 7155.24, 6468.95, 5719.50, 
                   6866.16, 8509.43, 9922.54, 9749.23, 9593.83, 8741.28])
hrvv = hrvv_o / tot_forest_area

# Read in wood density for each different candidate
# Only obtain parameter values from candidates

# Open and collect wood density information from candidates within the ensemble
# Path to all members
wd = []
param_prefix = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/mx_breeding_candidates254/fates_params_sr_ens_'
for i in np.arange(0,nens):
    ens_num = str(i+1).zfill(4)
    fpath = param_prefix+ens_num+'.nc'
    dtc = nc4.Dataset(fpath)
    tmp = dtc['fates_wood_density'][:]
    wd.append(tmp[0])
    dtc.close()

# from g cm-3 to kg m-3
arr_wd = 1000.0 * np.array(wd)

del tmp

# Calculate the harvested C amount
hrvc = []
for i in np.arange(0,nens):
    arr_hrvc = hrvv * arr_wd[i] * 0.5 / tot_forest_area
    hrvc.append(arr_hrvc)

# # Create land use dataset for each candidate (C-based)
# fpath = '/pscratch/sd/s/sshu3/FATES_MRV/fates_mxbreeding254_spinup_2024-05-27_e9515ed7a8_664db78d/run/'
# for i in np.arange(0,nens):
#     ens_num = str(i+1).zfill(4)
#     # Note: Time dimension in base land use timeseries need to be the recording dimension
#     tfile = 'landuse.timeseries_0.125x0.125_hist_simyr1850-2028.SanRafael.c231020_'+ens_num+'.nc'
#     cmd_line = 'cp '+fpath+'landuse.timeseries_0.125x0.125_hist_simyr1850-2028.SanRafael.c231020.nc '+fpath+tfile
#     os.system(cmd_line)
#     dtc = nc4.Dataset(fpath+tfile, 'r+', clobber=True)
#     # tmp=dtc['HARVEST_SH1'][:]
#     # ndim = len(np.shape(tmp))
#     dtc['HARVEST_SH1'][159:179,0,0] = hrvc[i][:]
#     dtc['HARVEST_SH2'][159:179,0,0] = np.zeros((20))
#     dtc['HARVEST_SH3'][159:179,0,0] = np.zeros((20))
#     dtc['HARVEST_VH1'][159:179,0,0] = np.zeros((20))
#     dtc['HARVEST_VH2'][159:179,0,0] = np.zeros((20))
#     dtc['GRAZING'][159:179,0,0] = np.zeros((20))
#     dtc['YEAR'][159:179] = lu_year[:]
#     dtc['time'][159:179] = lu_year[:]
#     dtc.close()
    
# # Create land use dataset for each candidate (A-based)
# fpath = '/pscratch/sd/s/sshu3/FATES_MRV/fates_mxbreeding254_spinup_2024-06-12_e9515ed7a8_664db78d/run/'
# tfile = 'landuse.timeseries_0.125x0.125_hist_simyr1850-2028.SanRafael.c231020.nc'
# dtc = nc4.Dataset(fpath+tfile, 'r+', clobber=True)
# dtc['HARVEST_SH1'][159:179,0,0] = hrva_o[:]
# dtc['HARVEST_SH2'][159:179,0,0] = np.zeros((20))
# dtc['HARVEST_SH3'][159:179,0,0] = np.zeros((20))
# dtc['HARVEST_VH1'][159:179,0,0] = np.zeros((20))
# dtc['HARVEST_VH2'][159:179,0,0] = np.zeros((20))
# dtc['GRAZING'][159:179,0,0] = np.zeros((20))
# dtc['YEAR'][159:179] = lu_year[:]
# dtc['time'][159:179] = lu_year[:]
# dtc.close()

### Draft Space / Testbed

In [ ]:
tmp = scaled_sample[0:50,:]

df  = pd.DataFrame(data=tmp, columns = param_names)

In [5]:
# NL [3.376, 5.064]: #3, #5, #6, #8, #9, #10
# BL [3.976, 5.964]: #5, #7, #12, #13, #17, #19, #20
# Exchange their correponding PFT specific parameters and form the following cases.
# If we ignore #5, we will have the following matrix
#      3    6    8    9    10  [NL parameters]
#  7  (1)  (2)  (3)  (4)  (5)
# 12  (6)  (7)  (8)  (9) (10)
# 13 (11) (12) (13) (14) (15)
# 17 (16) (17) (18) (19) (20)
# 19 (21) (22) (23) (24) (25)
# 20 (26) (27) (28) (29) (30)
# [BL parameters]
candidates = np.array([18, 135, 267, 291, 331, 352, 386, 420, 463, 471, 519, 541, 567, 834, 
                       883, 916, 938, 994, 1006, 1068, 1241, 1246, 1374, 1431, 1500, 1530, 1580])
nl_candidates = np.array([candidates[2], candidates[5], candidates[7], candidates[8], candidates[9]])
bl_candidates = np.array([candidates[6], candidates[11], candidates[12], candidates[16], candidates[18], candidates[19]])

### Examine parameters from failed cases

In [32]:
failed_cases_path = '/global/homes/s/sshu3/FATES_MRV/parameter_file_sandbox/sr_calibrated_candidates254/failed_cases/'
cmd_line = 'ls -1 '+failed_cases_path
flist = os.popen(cmd_line).read().split('\n')
# Somehow the last line is empty
del flist[-1]

# parameters
wd = []
vcmax_bl = []
vcmax_nl = []
sla_bl = []
sla_nl = []

for i in np.arange(0,len(flist)):
    fpath = failed_cases_path+flist[i]
    dtc = nc4.Dataset(failed_cases_path+flist[i])
    tmp = dtc['fates_wood_density'][:]
    wd.append(tmp[0])
    del tmp
    tmp = dtc['fates_leaf_vcmax25top'][:]
    vcmax_bl.append(tmp[0][0])
    vcmax_nl.append(tmp[0][1])
    del tmp
    tmp = dtc['fates_leaf_slatop'][:]
    sla_bl.append(tmp[0])
    sla_nl.append(tmp[1])
    del tmp
    
    dtc.close()